# Scenario:
## ==========================================================
## HUGGING FACE FINE-TUNING CODE
## Fine-tune any model and upload to Hugging Face Hub
## ==========================================================

In [5]:
# STEP 1: INSTALLATION AND SETUP
print("Installing required libraries...")
!pip install -q transformers datasets accelerate evaluate
!pip install -q huggingface_hub
!pip install -q peft bitsandbytes
!pip install -q torch torchvision torchaudio

Installing required libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━

In [6]:
# Import version check
import transformers
from packaging import version

In [7]:
# Import all required libraries
import torch
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict, load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
import evaluate
from google.colab import userdata
from huggingface_hub import login
import warnings
warnings.filterwarnings("ignore")

print("Libraries installed successfully!")

Libraries installed successfully!


In [8]:
# Check GPU
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

CUDA available: True
GPU: Tesla T4


# =============================
# STEP 2: CONFIGURATION
# =============================


In [9]:
# Model Configuration
MODEL_NAME = "distilbert-base-uncased"
TASK_TYPE = "classification"
NUM_LABELS = 2  # For binary classification (positive/negative)

In [10]:
# Training Configuration
MAX_LENGTH = 512
BATCH_SIZE = 8  # Reduce if you get memory errors
NUM_EPOCHS = 3
LEARNING_RATE = 2e-5
WARMUP_STEPS = 500

In [11]:
# Hugging Face Repository (CHANGE THIS!)
HF_USERNAME = "imrgurmeet"  # Replace with your HF username
MODEL_REPO_NAME = "fine-tuned-sentiment-model"  # Choose your model name
HF_REPO_NAME = f"{HF_USERNAME}/{MODEL_REPO_NAME}"

In [13]:
# Dataset Configuration
USE_CUSTOM_DATA = False  # Set to True if using custom data
DATASET_NAME = "stanfordnlp/imdb"  # Hugging Face dataset name

print(f"    Configuration:")
print(f"   Model: {MODEL_NAME}")
print(f"   Task: {TASK_TYPE}")
print(f"   Repository: {HF_REPO_NAME}")
print(f"   Dataset: {DATASET_NAME}")

    Configuration:
   Model: distilbert-base-uncased
   Task: classification
   Repository: imrgurmeet/fine-tuned-sentiment-model
   Dataset: stanfordnlp/imdb


# ===================================================
# STEP 3: HUGGING FACE AUTHENTICATION
# ===================================================

In [ ]:
print("Authenticating with Hugging Face...")

# Method 1: Using Colab Secrets (Recommended)
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(HF_TOKEN)
    print("Successfully logged in using Colab secrets!")
except Exception as e:
    print("Colab secrets not found. Using manual login...")
    # Method 2: Manual login
    try:
        login()
        print("Successfully logged in manually!")
    except Exception as e:
        print("Login failed. Please check your token.")
        print("To fix: Go to https://huggingface.co/settings/tokens")
        print("Create a token with 'write' permissions")
        print("Add it to Colab secrets as 'HF_TOKEN'")

# ==============================================
# STEP 4: LOAD MODEL AND TOKENIZER
# ==============================================

In [ ]:
print(f"\n Loading model and tokenizer: {MODEL_NAME}")

def load_model_and_tokenizer(model_name, num_labels, task_type):
    """Load model and tokenizer based on task type"""

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Add padding token if it doesn't exist
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    if task_type == "classification":
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )
    elif task_type == "text-generation":
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )
    else:
        raise ValueError("task_type must be 'classification' or 'text-generation'")

    return model, tokenizer

try:
    model, tokenizer = load_model_and_tokenizer(MODEL_NAME, NUM_LABELS, TASK_TYPE)
    print(f"Model and tokenizer loaded successfully!")
    print(f"Model parameters: {model.num_parameters():,}")
except Exception as e:
    print(f"Error loading model: {e}")


# ==============================================
# STEP 5: DATASET PREPARATION
# ==============================================

In [ ]:
print(f"\n Preparing dataset...")

def load_and_prepare_dataset():
    """Load and prepare dataset"""

    if USE_CUSTOM_DATA:
        # Option 1: Custom data - Modify this section for your data
        print("Using custom dataset...")

        # Example custom data (replace with your data)
        texts = [
            "This movie is absolutely fantastic! Great acting and plot.",
            "Terrible film, complete waste of time. Very disappointing.",
            "Amazing cinematography and brilliant storytelling. Loved it!",
            "Boring and predictable. Couldn't wait for it to end.",
            "Outstanding performance by all actors. Highly recommended!",
            "Poor script and bad direction. One of the worst movies ever."
        ]
        labels = [1, 0, 1, 0, 1, 0]  # 1 = positive, 0 = negative

        # Create dataset
        data_dict = {"text": texts, "label": labels}
        dataset = Dataset.from_dict(data_dict)

        # Split into train/test
        dataset = dataset.train_test_split(test_size=0.2, seed=42)

    else:
        # Option 2: Load from Hugging Face Hub
        print(f"Loading dataset from Hugging Face: {DATASET_NAME}")
        dataset = load_dataset(DATASET_NAME)

        # Use smaller subset for faster training (optional)
        if len(dataset["train"]) > 5000:
            dataset["train"] = dataset["train"].shuffle(seed=42).select(range(5000))
        if "test" in dataset and len(dataset["test"]) > 1000:
            dataset["test"] = dataset["test"].shuffle(seed=42).select(range(1000))

    return dataset

try:
    dataset = load_and_prepare_dataset()
    print(f"Dataset loaded successfully!")
    print(f"Train samples: {len(dataset['train'])}")
    if "test" in dataset:
        print(f"Test samples: {len(dataset['test'])}")

    # Show sample data
    print("\n Sample data:")
    for i in range(min(3, len(dataset["train"]))):
        sample = dataset["train"][i]
        print(f"Text: {sample['text'][:100]}...")
        print(f"Label: {sample['label']}")
        print()

except Exception as e:
    print(f"Error loading dataset: {e}")

# ===================================
# STEP 6: TOKENIZATION
# ===================================

In [ ]:
print("Tokenizing dataset...")

def tokenize_function(examples):
    """Tokenize the input texts"""
    return tokenizer(
        examples["text"],
        truncation=True,
        padding=True,
        max_length=MAX_LENGTH,
        return_tensors="pt"
    )

try:
    # Apply tokenization
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    print("Tokenization completed!")

    # Show tokenized sample
    sample = tokenized_dataset["train"][0]
    print(f"Sample input_ids shape: {len(sample['input_ids'])}")
    print(f"Sample tokens: {tokenizer.convert_ids_to_tokens(sample['input_ids'][:10])}")

except Exception as e:
    print(f"Error during tokenization: {e}")

# ====================================================
# STEP 7: SETUP PARAMETER EFFICIENT FINE-TUNING
# ====================================================

In [24]:
#printing out the model’s architecture and inspect the layer names: (can skip also)
for name, module in model.named_modules():
    print(name)


distilbert
distilbert.embeddings
distilbert.embeddings.word_embeddings
distilbert.embeddings.position_embeddings
distilbert.embeddings.LayerNorm
distilbert.embeddings.dropout
distilbert.transformer
distilbert.transformer.layer
distilbert.transformer.layer.0
distilbert.transformer.layer.0.attention
distilbert.transformer.layer.0.attention.dropout
distilbert.transformer.layer.0.attention.q_lin
distilbert.transformer.layer.0.attention.k_lin
distilbert.transformer.layer.0.attention.v_lin
distilbert.transformer.layer.0.attention.out_lin
distilbert.transformer.layer.0.sa_layer_norm
distilbert.transformer.layer.0.ffn
distilbert.transformer.layer.0.ffn.dropout
distilbert.transformer.layer.0.ffn.lin1
distilbert.transformer.layer.0.ffn.lin2
distilbert.transformer.layer.0.ffn.activation
distilbert.transformer.layer.0.output_layer_norm
distilbert.transformer.layer.1
distilbert.transformer.layer.1.attention
distilbert.transformer.layer.1.attention.dropout
distilbert.transformer.layer.1.attention.q

In [27]:
print("Setting up Parameter Efficient Fine-Tuning (LoRA)...")

USE_PEFT = True  # Set to False for full fine-tuning

if USE_PEFT:
    try:
        # LoRA configuration
        peft_config = LoraConfig(
            r=16,  # rank
            lora_alpha=32,  # scaling factor
            lora_dropout=0.1,
            bias="none",
            task_type="SEQ_CLS",
            # target_modules = ["q_lin", "k_lin", "v_lin", "out_lin"]    #if want full model fine-tuning comment this line
        )

        # Apply PEFT
        model = prepare_model_for_kbit_training(model)
        model = get_peft_model(model, peft_config)

        print(f"PEFT (LoRA) applied successfully!")
        print(f"Trainable parameters: {model.num_parameters(only_trainable=True):,}")
        print(f"Total parameters: {model.num_parameters():,}")
        print(f"Trainable %: {100 * model.num_parameters(only_trainable=True) / model.num_parameters():.2f}%")

    except Exception as e:
        print(f"PEFT setup failed, using full fine-tuning: {e}")
        USE_PEFT = False

if not USE_PEFT:
    print("Using full model fine-tuning")

Setting up Parameter Efficient Fine-Tuning (LoRA)...
PEFT setup failed, using full fine-tuning: Please specify `target_modules` or `target_parameters`in `peft_config`
Using full model fine-tuning


# =================================
# STEP 8: TRAINING SETUP
# =================================

In [28]:
print("\n Setting up training configuration...")

print(f"Transformers version: {transformers.__version__}")

# Determine parameter names based on version
if version.parse(transformers.__version__) >= version.parse("4.20.0"):
    eval_strategy_param = "eval_strategy"
else:
    eval_strategy_param = "evaluation_strategy"

print(f"Using parameter: {eval_strategy_param}")



 Setting up training configuration...
Transformers version: 4.55.1
Using parameter: eval_strategy


In [29]:
# Create training arguments
training_kwargs = {
    "output_dir": "./results",
    "num_train_epochs": NUM_EPOCHS,
    "per_device_train_batch_size": BATCH_SIZE,
    "per_device_eval_batch_size": BATCH_SIZE,
    "warmup_steps": WARMUP_STEPS,
    "weight_decay": 0.01,
    "learning_rate": LEARNING_RATE,
    "logging_dir": "./logs",
    "logging_steps": 100,
    "save_steps": 500,
    "save_total_limit": 2,
    "dataloader_pin_memory": False,
    "fp16": torch.cuda.is_available(),
}


In [30]:
# Add evaluation parameters only if test set exists
if "test" in tokenized_dataset:
    training_kwargs.update({
        eval_strategy_param: "steps",
        "eval_steps": 500,
        "load_best_model_at_end": True,
        "metric_for_best_model": "eval_accuracy",
        "greater_is_better": True,
    })

In [31]:
# Add other optional parameters if supported
try:
    training_kwargs.update({
        "report_to": "none",
        "push_to_hub": False,
    })
except:
    pass

In [32]:
# Create TrainingArguments
training_args = TrainingArguments(**training_kwargs)

print("Training arguments created successfully!")

Training arguments created successfully!


In [33]:
# Metrics function with fallback
def compute_metrics(eval_pred):
    """Compute metrics for evaluation with fallback"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    try:
        # Try using evaluate library
        accuracy_metric = evaluate.load("accuracy")
        return accuracy_metric.compute(predictions=predictions, references=labels)
    except:
        # Fallback to manual calculation
        accuracy = (predictions == labels).mean()
        return {"accuracy": float(accuracy)}

In [34]:
# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Initialize trainer
trainer_kwargs = {
    "model": model,
    "args": training_args,
    "train_dataset": tokenized_dataset["train"],
    "tokenizer": tokenizer,
    "data_collator": data_collator,
}

In [35]:
# Add evaluation dataset and metrics if available
if "test" in tokenized_dataset:
    trainer_kwargs.update({
        "eval_dataset": tokenized_dataset["test"],
        "compute_metrics": compute_metrics,
    })

In [36]:
# Create trainer
trainer = Trainer(**trainer_kwargs)

In [37]:
# Add callbacks if supported
if "test" in tokenized_dataset:
    try:
        from transformers import EarlyStoppingCallback
        trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=3))
        print("Early stopping callback added")
    except Exception as e:
        print(f"Early stopping not available: {e}")

print("Training setup completed!")

Early stopping callback added
Training setup completed!


# ================================
# STEP 9: START TRAINING
# ================================

In [40]:
print("Starting training...")
print(f"Training samples: {len(tokenized_dataset['train'])}")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")

try:
    # Start training
    training_results = trainer.train()

    print("Training completed successfully!")
    print(f"Final training loss: {training_results.training_loss:.4f}")

    # Evaluate if test set available
    if "test" in tokenized_dataset:
        print("\n Evaluating model...")
        eval_results = trainer.evaluate()
        print(f"Evaluation completed!")
        for key, value in eval_results.items():
            if isinstance(value, (int, float)):
                print(f"{key}: {value:.4f}")

except Exception as e:
    print(f"Training failed: {e}")

Starting training...
Training samples: 5000
Epochs: 3
Batch size: 8
Learning rate: 2e-05
Training failed: element 0 of tensors does not require grad and does not have a grad_fn


# =================================
# STEP 10: TEST THE MODEL
# =================================

In [41]:
print("Testing the fine-tuned model...")

def test_model_predictions(texts_to_test):
    """Test the model with sample inputs"""
    results = []

    for text in texts_to_test:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

        # Move to same device as model
        if torch.cuda.is_available():
            inputs = {k: v.cuda() for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_class = torch.argmax(predictions, dim=-1).item()
            confidence = torch.max(predictions).item()

        results.append({
            "text": text,
            "predicted_class": predicted_class,
            "confidence": confidence,
            "label": "positive" if predicted_class == 1 else "negative"
        })

    return results

# Test samples
test_samples = [
    "This movie is absolutely fantastic! I loved every minute of it.",
    "Terrible film, complete waste of time. Very disappointing.",
    "It was okay, nothing special but not bad either.",
    "Amazing cinematography and brilliant acting. Highly recommended!",
    "Boring and predictable storyline. Couldn't finish watching."
]

try:
    test_results = test_model_predictions(test_samples)

    print("Test Results:")
    for result in test_results:
        print(f"   Text: '{result['text'][:60]}...'")
        print(f"   Prediction: {result['label']} (confidence: {result['confidence']:.3f})")
        print()

except Exception as e:
    print(f"Testing failed: {e}")


Testing the fine-tuned model...
Test Results:
   Text: 'This movie is absolutely fantastic! I loved every minute of ...'
   Prediction: positive (confidence: 0.504)

   Text: 'Terrible film, complete waste of time. Very disappointing....'
   Prediction: negative (confidence: 0.533)

   Text: 'It was okay, nothing special but not bad either....'
   Prediction: negative (confidence: 0.507)

   Text: 'Amazing cinematography and brilliant acting. Highly recommen...'
   Prediction: negative (confidence: 0.517)

   Text: 'Boring and predictable storyline. Couldn't finish watching....'
   Prediction: negative (confidence: 0.517)



# =======================================
# STEP 11: SAVE MODEL LOCALLY
# =======================================

In [42]:
print("Saving model locally...")

save_directory = "./fine-tuned-model"

try:
    # Save model and tokenizer
    if USE_PEFT:
        # For PEFT models, save the adapter
        model.save_pretrained(save_directory)
    else:
        # For full fine-tuning
        model.save_pretrained(save_directory)

    tokenizer.save_pretrained(save_directory)

    print(f"Model saved to {save_directory}")

    # List saved files
    import os
    files = os.listdir(save_directory)
    print(f"   Saved files: {files}")

except Exception as e:
    print(f"Error saving model: {e}")

Saving model locally...
Model saved to ./fine-tuned-model
   Saved files: ['adapter_model.safetensors', 'vocab.txt', 'tokenizer_config.json', 'tokenizer.json', 'special_tokens_map.json', 'README.md', 'adapter_config.json']


# ===================================
# STEP 12: CREATE MODEL CARD
# ===================================

In [41]:
model_card_content = f"""---
language: en
tags:
- fine-tuned
- {MODEL_NAME}
- text-classification
- sentiment-analysis
license: apache-2.0
datasets:
- {DATASET_NAME}
---

# Fine-tuned {MODEL_NAME} for Sentiment Analysis

This model is a fine-tuned version of [{MODEL_NAME}](https://huggingface.co/{MODEL_NAME}) on the {DATASET_NAME} dataset.

## Model Details

- **Base Model**: {MODEL_NAME}
- **Task**: Text Classification (Sentiment Analysis)
- **Dataset**: {DATASET_NAME}
- **Fine-tuning Method**: {"PEFT (LoRA)" if USE_PEFT else "Full Fine-tuning"}
- **Training Samples**: {len(tokenized_dataset['train'])}
- **Epochs**: {NUM_EPOCHS}
- **Batch Size**: {BATCH_SIZE}
- **Learning Rate**: {LEARNING_RATE}

## Usage

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("{HF_REPO_NAME}")
model = AutoModelForSequenceClassification.from_pretrained("{HF_REPO_NAME}")

Example usage
text = "This movie is amazing! I really enjoyed watching it."
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predicted_class = torch.argmax(predictions, dim=-1).item()
confidence = torch.max(predictions).item()

0 = negative, 1 = positive
sentiment = "positive" if predicted_class == 1 else "negative"
print(f"Sentiment: {{sentiment}} (confidence: {{confidence:.3f}})")

## Training Details

- **Optimizer**: AdamW
- **Weight Decay**: 0.01
- **Warmup Steps**: {WARMUP_STEPS}
- **Mixed Precision**: {"Enabled" if torch.cuda.is_available() else "Disabled"}

## Performance

The model was trained and evaluated on the {DATASET_NAME} dataset. Please refer to the training logs for detailed performance metrics.

## Intended Use

This model is intended for sentiment analysis of English text. It classifies text into two categories:
- 0: Negative sentiment
- 1: Positive sentiment

## Limitations

- The model is trained primarily on movie reviews and may not generalize well to other domains
- Performance may vary on text with mixed or neutral sentiments
- The model is designed for English text only

## Training Infrastructure

- **Platform**: Google Colab
- **GPU**: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"}
- **Framework**: PyTorch + Transformers

---

*This model was fine-tuned using the Hugging Face Transformers library.*
"""

try:
    # Save model card
    with open(f"{save_directory}/README.md", "w") as f:
        f.write(model_card_content)

    print("Model card created successfully!")

except Exception as e:
    print(f"Error creating model card: {e}")

Model card created successfully!


# =============================================
# STEP 13: UPLOAD TO HUGGING FACE HUB
# =============================================

In [ ]:
print(f"Uploading to Hugging Face Hub: {HF_REPO_NAME}")
print("This may take a few minutes...")

try:
    # Method 1: Direct push (recommended)
    if USE_PEFT:
        # For PEFT models, push the adapter
        model.push_to_hub(HF_REPO_NAME, private=False)
    else:
        # For full models
        model.push_to_hub(HF_REPO_NAME, private=False)

    tokenizer.push_to_hub(HF_REPO_NAME, private=False)

    print("Model uploaded successfully!")
    print(f"Your model is available at: https://huggingface.co/{HF_REPO_NAME}")
    print(f"Model card: https://huggingface.co/{HF_REPO_NAME}/blob/main/README.md")

except Exception as e:
    print(f"Upload failed: {e}")
    print("Trying alternative upload method...")

    try:
        # Method 2: Manual upload
        from huggingface_hub import HfApi, create_repo

        # Create repository
        create_repo(HF_REPO_NAME, exist_ok=True, private=False)

        # Upload files
        api = HfApi()
        api.upload_folder(
            folder_path=save_directory,
            repo_id=HF_REPO_NAME,
            repo_type="model"
        )

        print("Alternative upload successful!")
        print(f"My model path: https://huggingface.co/{HF_REPO_NAME}")

    except Exception as e2:
        print(f"Alternative upload also failed: {e2}")
        print("Manual steps to upload:")
        print(f"   1. Go to https://huggingface.co/new")
        print(f"   2. Create repository: {HF_REPO_NAME}")
        print(f"   3. Upload files from: {save_directory}")

# =============================================
# STEP 14: VERIFICATION AND TESTING
# =============================================

In [ ]:
print(f"\n Verifying uploaded model...")

try:
    # Try to load the uploaded model
    from transformers import AutoTokenizer, AutoModelForSequenceClassification

    print("Loading model from Hub...")
    uploaded_tokenizer = AutoTokenizer.from_pretrained(HF_REPO_NAME)
    uploaded_model = AutoModelForSequenceClassification.from_pretrained(HF_REPO_NAME)

    print("Model loaded successfully from Hub!")

    # Test the uploaded model
    test_text = "This is an amazing product! Highly recommend it."
    inputs = uploaded_tokenizer(test_text, return_tensors="pt")

    with torch.no_grad():
        outputs = uploaded_model(**inputs)
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(prediction).item()

    print(f"Test prediction: {'positive' if predicted_class == 1 else 'negative'}")

except Exception as e:
    print(f"Could not verify uploaded model: {e}")
    print("The model might still be processing on the Hub")

# ==========================
# FINAL SUMMARY
# ==========================

In [44]:
# Full summary of the fine-tuned model
print("\n" + "="*60)
print("FINE-TUNING COMPLETED SUCCESSFULLY!")
print("="*60)
print(f"Training Summary:")
print(f"• Model: {MODEL_NAME}")
print(f"• Task: {TASK_TYPE}")
print(f"• Method: {'PEFT (LoRA)' if USE_PEFT else 'Full Fine-tuning'}")
print(f"• Dataset: {DATASET_NAME}")
print(f"• Training samples: {len(tokenized_dataset['train'])}")
print(f"• Epochs: {NUM_EPOCHS}")
print(f"• Batch size: {BATCH_SIZE}")

print(f"My Model:")
print(f"• Repository: {HF_REPO_NAME}")
print(f"• URL: https://huggingface.co/{HF_REPO_NAME}")
print(f"• Local path: {save_directory}")

print(f"\n Next Steps:")
print(f"1. Test your model on new data")
print(f"2. Share it with the community")
print(f"3. Use it in your applications")
print(f"4. Continue fine-tuning if needed")

print(f"\n Usage in your code:")
print(f'''
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("{HF_REPO_NAME}")
model = AutoModelForSequenceClassification.from_pretrained("{HF_REPO_NAME}")

text = "Your text here"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)
''')

print("="*60)
print("Happy fine-tuning! Your model is ready to use!")
print("="*60)


FINE-TUNING COMPLETED SUCCESSFULLY!
Training Summary:
• Model: distilbert-base-uncased
• Task: classification
• Method: Full Fine-tuning
• Dataset: stanfordnlp/imdb
• Training samples: 5000
• Epochs: 3
• Batch size: 8

 Your Model:
• Repository: imrgurmeet/fine-tuned-sentiment-model
• URL: https://huggingface.co/imrgurmeet/fine-tuned-sentiment-model
• Local path: ./fine-tuned-model

 Next Steps:
1. Test your model on new data
2. Share it with the community
3. Use it in your applications
4. Continue fine-tuning if needed

 Usage in your code:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("imrgurmeet/fine-tuned-sentiment-model")
model = AutoModelForSequenceClassification.from_pretrained("imrgurmeet/fine-tuned-sentiment-model")

text = "Your text here"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
prediction = torch.nn.functional.softmax(outputs.logits, dim=-1)

Happy fine-tuning! Your mode

# ========================================================
# COMPLETE INFERENCE CODE FOR FINE-TUNED MODEL
# Load and use your model for predictions
# ========================================================

# ================================================
# Model: imrgurmeet/fine-tuned-sentiment-model
# =================================================

In [3]:
# SIMPLE VERSION:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load once
tokenizer = AutoTokenizer.from_pretrained("imrgurmeet/fine-tuned-sentiment-model")
model = AutoModelForSequenceClassification.from_pretrained("imrgurmeet/fine-tuned-sentiment-model")

# Use anywhere:
def analyze(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=-1).item()
    return "positive" if prediction == 1 else "negative"

# Test it:
print(analyze("I love this!"))      # positive
print(analyze("This is terrible!")) # negative
print(analyze("It's okay"))         # positive or negative


positive
positive
positive
